<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

## **MASTERGIS: Clasificacion de Imagenes Satelitales**

En esta lectura, aprenderemos sobre:

- **Clasificacion supervisada**.
- **Clasificacion no supervisada**.
<center>
<img src='https://user-images.githubusercontent.com/16768318/73627817-0a23e100-4646-11ea-83f4-07293b85c4d9.png' width = 50% align="right">

<img src='https://user-images.githubusercontent.com/16768318/73627849-2aec3680-4646-11ea-9dd3-b52b13652833.gif' width = 30% align="left">
</center>

https://www.mdpi.com/2072-4292/11/23/2881


Se recomienda utilizar otras librerias en python para hacer los modelos de ML, como sckit-learn, tensor flow, keras y otros, sin embargo, en GEE tambien se pueden generar modelos pero son muy limitados. Asi que lo mejor es realizar el modelo en local en python, y luego subir el modelo para predecir en gee.

In [1]:
#@title Credenciales Google Earth Engine
import ee
ee.Authenticate()
ee.Initialize(project = 'ee-jtantaroman')

In [2]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **1. Clasificación supervisada**
El modulo **ee.Classifier** maneja la clasificacion supervisada en Earth Engine. El flujo de trabajo general para la clasificacion es:

1. Recopilar datos de entrenamiento. Reuna puntos que tienen como propiedad el **"target"** y las variables predictoras.

2. Instanciar un clasificador. Establezca sus parametros si es necesario.

3. Entrene al clasificador utilizando los datos de entrenamiento.

4. Clasificar una imagen o FeatureCollection.

5. Estime el error de clasificacion con datos de validacion independientes.


Los datos de entrenamiento siempre deben definirse como un **ee.FeatureCollection** con una propiedad que almacene el target y propiedades que almacenen las variables predictoras. El **target** debe ser enteros pequeños, consecutivos y no negativos que representen clases. Si es necesario, utilice **remap()** para convertir los valores de clase en enteros consecutivos a partir de 0. Los predictores deben ser numericos.

Obtenga un clasificador de uno de los constructores en **ee.Classifier**. Entrene al clasificador usando **classifier.train()**. Clasifique una imagen o FeatureCollection usando **classify()**. El siguiente ejemplo utiliza un clasificador de arboles de clasificacion y regresion (CART) (Breiman et al. 1984) para predecir areas forestales y no forestales en la Amazonia:

In [15]:
# Cargar puntos de entrenamiento. La propiedad numerica 'clase' almacena etiquetas conocidas
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# Las imagenes de entrada son un compuesto Landsat 8 sin nubes.
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
       .filterBounds(points)

# Filtrar las imagenes por score de nubes y percentiles (% de nubes que presenta la imagen)
# Use the 'quality band' to mask out cloudy pixels before compositing
image = l8.filterDate('2018-01-01', '2018-12-31') \
    .map(lambda img: img.updateMask(img.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0))) \
    .median() \
    .clip(points.geometry().bounds())

# Use estas bandas para la prediccion.
bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']

**Visualizemos los datos de entrada**

In [16]:
center = points.geometry().centroid().coordinates().getInfo()
print(points.first().getInfo())
mapdisplay(center,{'train_dataset':points.getInfo(),
                   'image':image.getMapId({'bands':['SR_B5','SR_B4','SR_B3'],'gamma':1.4})})

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-122.27096557617189, 37.820452055421086]}, 'id': '00000000000000000003', 'properties': {'landcover': 0}}


**Clasificar una imagen landsat 8 usando arboles de decision (CART)**

In [19]:
# Esta propiedad de la tabla almacena el target (cobertura del suelo).
label = 'landcover'

# Superponga los puntos en las imagenes para obtener los datos de entrenamiento.
# Generando la data para entrenar
training = image.select(bands).sampleRegions( #seleccion de variables predictoras X
  collection = points, # el target  o Y
  properties = [label], # el nombre del Y
  scale = 30)
training.first().getInfo()

# Entrena un modelo clasificador CART con parámetros predeterminados.
trained = ee.Classifier.smileCart().train(training, label, bands)

# Clasifique la imagen con las mismas bandas utilizadas para el entrenamiento.
# Ahora el modelo clasificara con lo que ha entrenado, el resto de la imagen
classified = image.select(bands).classify(trained)

In [21]:
mapdisplay(center,{'train_dataset':points.getInfo(),
                   'image':image.getMapId({'bands':['SR_B5','SR_B4','SR_B3'],'gamma':1.4}),
                   'clasificacion':classified.getMapId({'min': 0, 'max': 2, 'palette': ['red', 'green', 'blue']})})

In [22]:
#Esto se debe hacer con una base aparte no la de entrenamiento, a esta se llama base de testeo
train_accuracy = trained.confusionMatrix()

print('Validation error matrix: ', train_accuracy.getInfo())
print('Validation overall accuracy: ', train_accuracy.accuracy().getInfo())

Validation error matrix:  [[32, 0, 0], [0, 31, 0], [0, 0, 34]]
Validation overall accuracy:  1


Este siguiente ejemplo utiliza un clasificador **Random Forest** con 10 arboles ([Breiman 2001](https://rd.springer.com/article/10.1023/A:1010933404324)). Para evaluar la precision de un clasificador, utilizamos una **matriz de confusion**(Stehman 1997).

In [34]:
# Las imagenes de entrada son un compuesto Landsat 8 sin nubes.
center = [-62.836, -9.2399]
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
       .filterBounds(ee.Geometry.Point(center))

image = l8.filterDate('2018-01-01', '2018-12-31') \
    .map(lambda img: img.updateMask(img.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0))) \
    .median()


# Use estas bandas para la prediccion.
bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10'];

# Poligonos creados manualmente.
# Estos poligonos van a servir como nuestra base de datos con target (Y), y GEE tomara puntos aleatorios dentro de estos poligonos con sus respectivas clases (Y)
forest1 = ee.Geometry.Rectangle(-63.0187, -9.3958, -62.9793, -9.3443)
forest2 = ee.Geometry.Rectangle(-62.8145, -9.206, -62.7688, -9.1735)
nonForest1 = ee.Geometry.Rectangle(-62.8161, -9.5001, -62.7921, -9.4486)
nonForest2 = ee.Geometry.Rectangle(-62.6788, -9.044, -62.6459, -8.9986)

# Hacer un FeatureCollection de las geometrias hechas a mano.
polygons = ee.FeatureCollection([
  ee.Feature(nonForest1, {'class': 0}),
  ee.Feature(nonForest2, {'class': 0}),
  ee.Feature(forest1, {'class': 1}),
  ee.Feature(forest2, {'class': 1}),
])

# Obtenga los valores para todos los pixeles en cada poligono en el entrenamiento.
# Obtenga la muestra de los poligonos FeatureCollection.
# Mantenga esta lista de propiedades de los poligonos.
# Establezca la escala para obtener pixeles Landsat en los poligonos.
training = image.sampleRegions(
  collection = polygons,
  properties = ['class'],  #como aca hay una propiedad no es necesario colocarla
  scale = 30
)


# Crear un clasificador RF con parámetros personalizados.
classifier = ee.Classifier.smileRandomForest(10)

# Entrenar el clasificador
trained = classifier.train(training, 'class', bands)

# Clasificar la imagen
classified = image.classify(trained)

In [36]:
mapdisplay(center, {'composite':image.getMapId({'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'gamma': 2},),
                    'deforestation': classified.getMapId({'min': 0, 'max': 1, 'palette': ['red', 'green']}),
                    'polygon':polygons.getInfo()})

In [37]:
train_accuracy = training.classify(trained)

# Obtenga una matriz de confusión que represente la precisión esperada.
trainAccuracy = train_accuracy.errorMatrix('class', 'classification')
print('Validation error matrix: ', trainAccuracy.getInfo())
print('Validation overall accuracy: ', trainAccuracy.accuracy().getInfo())


Validation error matrix:  [[37554, 1], [2, 48430]]
Validation overall accuracy:  0.999965111005152


### **2. Clasificacion no supervisada (clustering)**

El modulo **ee.Clusterer** maneja la clasificacion no supervisada (o agrupamiento) en Earth Engine. Estos algoritmos se basan actualmente en los algoritmos con el mismo nombre en [Weka](https://www.cs.waikato.ac.nz/ml/weka/).

Los clusteres se usan de la misma manera que los **clasificadores** en Earth Engine. El flujo de trabajo general para la agrupacion es el siguiente:

1. Ensamblar entidades con propiedades numericas en las que encontrar clusteres.

2. Instanciar un clusterer. Establezca sus parametros si es necesario.

3. Entrene al clusterer usando los datos de entrenamiento.

4. Aplique el clusterer a una colección de imagenes o caracteristicas.

5. Ponga nombre a los clusters.

Los datos de entrenamiento son un **ee.FeatureCollection** con propiedades que se ingresaran al clusterer. A diferencia de los clasificadores, **no hay un valor de clase de entrada para un ee.Clusterer**. Al igual que los clasificadores, se espera que los datos para el entrenamiento y los pasos de aplicacion tengan el mismo numero de valores. Cuando se aplica un cluster entrenado a una imagen o tabla, asigna una ID de cluster entero a cada pixel o feature.

In [38]:
# Cargue un compuesto Landsat previamente calculado para la entrada.
Input = ee.Image('LANDSAT/LE7_TOA_1YEAR/2001')

# Defina una region en la que generar una muestra de la entrada.
region = ee.Geometry.Rectangle(29.7, 30, 32.5, 31.7)
region_img = ee.Image().paint(region, 0, 2)

# Mostrar la region de muestra.
center = [31.0,31.5]
mapdisplay(center,{'Region':region_img.getMapId()},zoom_start=8)

In [39]:
# Hacer el conjunto de datos de entrenamiento. Sample es para no supervisada, porque no necesitas una variable Y aca
# Tomara 5mil puntos de toda la region que se muestra en la parte superior
training = Input.sample(
  region=region,
  scale=30,
  numPixels=5000
)

# Instanciar al clusterer y entrenarlo.
clusterer = ee.Clusterer.wekaKMeans(15).train(training)

# Agrupe la entrada utilizando el agrupador capacitado.
result = Input.cluster(clusterer)
result_tk = result.randomVisualizer().getMapId()

In [40]:
# Kmeans es mucho mas rapido porque es un modelo que utiliza la gradiente y es mas paralelizable en GEE a diferencia de random forest
# Mostrar los grupos con colores aleatorios.
mapdisplay(center, {'results':result_tk},zoom_start=8)

### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>